# Background

Deep-Learning Package Zoo
* Torch
* Caffe
* Theano (Keras, Lasagne)
* CuDNN
* Tensorflow 
* Mxnet
* etc.

### package design choices

* Model specification: Configuration file (e.g. Caffe, DistBelief, CNTK) 
* programmatic generation (e.g. Torch, Theano, Tensorflow)
    * choice of high-level language: Lua (Torch) vs. Python (Theano, Tensorflow) vs others.
    * python has rich community and library infrastructure.

### Theano VS TensorFlow

* Theano and TensorFlow are very similar systems
* TensorFlow has better support for distributed systems though, and has development funded by Google
* Theano is an academic project.

### Tensor Definition

In [12]:
from IPython.display import Image

![tensor_def](figs/tensor_def.png)

### tensorflow vs numpy

* TensorFlow and Numpy are quite similar. Both are N-d array libraries!
* Numpy has Ndarray support, but doesn’t offer methods to create tensor functions and automatically compute derivatives (+ no GPU support).

![np_vs_tf](figs/np_vs_tf.png)

### Exe01

TensorFlow computations define a computation graph that has no numerical value until evaluated.

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.4.1'

In [15]:
tf.compat.v1.disable_eager_execution()

In [16]:
# this is just a computation graph
ta = tf.zeros((2,2))
print(ta)

Tensor("zeros:0", shape=(2, 2), dtype=float32)


In [17]:
# init = tf.global_variables_initializer()   # create nodes in the graph, use in v1

# for v2.x
init = tf.compat.v1.global_variables_initializer()   # create nodes in the graph, use in v1

In [18]:
# sess = tf.Session() # v1
sess = tf.compat.v1.Session()

2022-11-08 00:12:56.672336: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:12:56.672465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:12:56.672750: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:12:56.672775: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:12:56.672794: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:12:56.672804: I tensorflow/stream_executor/pl

In [19]:
sess.run(init)  # initialize all the variables

v = sess.run(ta)  
print(v)

sess.close()  # close the session 

2022-11-08 00:14:26.015826: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2022-11-08 00:14:26.803517: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


[[0. 0.]
 [0. 0.]]


### Some background knowledge

* TensorFlow programs are usually structured into a construction phase, that assembles a graph, and an execution phase that uses a session to execute ops in the graph.

* A Session object encapsulates the environment in which Tensor objects are evaluated.

* All computations add nodes to global default graph


### more elegant implementation

In [23]:
ta = tf.zeros((2,2))
x = tf.Variable(3, name="x") # variable objects can be initialized from constants or random values
y = tf.Variable('hello', name="y")
z = tf.constant(5.0)
# R = tf.Variable(tf.random_normal((2,2)), name="random_weights")   # v1
R = tf.Variable(tf.random.normal((2,2)), name="random_weights")  

init = tf.compat.v1.global_variables_initializer()   # create nodes in the graph
with tf.compat.v1.Session() as sess:
    init.run()
    print(sess.run(ta))
    print(sess.run(x))
    print(str(sess.run(y)))
    print(sess.run(z))

2022-11-08 00:18:41.175020: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:18:41.175167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:18:41.175388: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:18:41.175443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:18:41.175481: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:18:41.175507: I tensorflow/stream_executor/pl

[[0. 0.]
 [0. 0.]]
3
b'hello'
5.0


### Why I should reset graph?

In Jupyter (or in a Python shell), it is common to run the same commands more than once while
you are experimenting. As a result, *** you may end up with a default graph containing many duplicate
nodes ***. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient
solution is to just reset the default graph by running tf.reset_default_graph().


## Exe02

What is the difference between the following two versions?

In [24]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.compat.v1.Session() as sess:
    init.run()
    print(y.eval())  # 10
    print(z.eval())  # 15
    #print(sess.run(y))
    #print(sess.run(z))

10
15


2022-11-08 00:19:24.925772: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:19:24.925911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:19:24.926147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:19:24.926174: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:19:24.926194: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:19:24.926204: I tensorflow/stream_executor/pl

In [25]:
with tf.compat.v1.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


2022-11-08 00:20:02.073406: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:20:02.073568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:20:02.073857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:20:02.073882: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:20:02.073901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:20:02.073910: I tensorflow/stream_executor/pl

### exe03

update tf variable in a for loop

In [27]:
state = tf.Variable(0, name="counter")
#new_value = state + 1
new_value = tf.add(state, tf.constant(1)) #  new_value = state + 1
update = tf.compat.v1.assign(state, new_value)  #  state = new_value

init = tf.compat.v1.global_variables_initializer()   # create nodes in the graph
with tf.compat.v1.Session() as sess:
    init.run()
    print(sess.run(state))
    for i in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


2022-11-08 00:21:02.573138: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:21:02.573287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:21:02.573550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:21:02.573582: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:21:02.573605: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:21:02.573616: I tensorflow/stream_executor/pl

### input numpy array to tf

In [29]:
import numpy as np

a = np.zeros((3,3))
ta = tf.convert_to_tensor(a)

init = tf.compat.v1.global_variables_initializer()   # create nodes in the graph
with tf.compat.v1.Session() as sess:
    init.run()
    print(sess.run(ta))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


2022-11-08 00:22:04.574027: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:22:04.574172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:22:04.574421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:22:04.574450: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:22:04.574471: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:22:04.574482: I tensorflow/stream_executor/pl

### placeholder and feed dictionaries

* Inputting data with tf.convert_to_tensor() is convenient, but doesn’t scale.
* Use tf.placeholder variables (dummy nodes that provide entry points for data to computational graph).
* A feed_dict is a python dictionary mapping from tf. placeholder vars (or their names) to data (numpy arrays, lists, etc.).
 

In [32]:
input1 = tf.compat.v1.placeholder(tf.float32)
input2 = tf.compat.v1.placeholder(tf.float32)

#output = tf.mul(input1, input2) # deprecated!
output = tf.multiply(input1, input2)


init = tf.compat.v1.global_variables_initializer()   # create nodes in the graph
with tf.compat.v1.Session() as sess:
    init.run()
    print(sess.run([output], feed_dict={input1:[1, 2,3,4], input2:[1, 2,3,4]}))  #   7 x 2 = 14

[array([ 1.,  4.,  9., 16.], dtype=float32)]


2022-11-08 00:22:48.948811: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 00:22:48.948964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 00:22:48.949234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.91GiB deviceMemoryBandwidth: 312.92GiB/s
2022-11-08 00:22:48.949259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 00:22:48.949277: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 00:22:48.949287: I tensorflow/stream_executor/pl

# References

[1] https://github.com/ageron/handson-ml

[2] https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb

[3] https://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf